In [2]:
import pyspark
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/04 23:23:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [39]:
df_fhv_2019_10 = spark.read \
.option("header", "true") \
.csv('fhv_2019_10')

In [40]:
df_fhv_2019_10.show(5)

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|dispatching_base_num|    pickup_datetime|   dropOff_datetime|PUlocationID|DOlocationID|SR_Flag|Affiliated_base_number|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|              B00009|2019-10-01 00:23:00|2019-10-01 00:35:00|         264|         264|   NULL|                B00009|
|              B00013|2019-10-01 00:11:29|2019-10-01 00:13:22|         264|         264|   NULL|                B00013|
|              B00014|2019-10-01 00:11:43|2019-10-01 00:37:20|         264|         264|   NULL|                B00014|
|              B00014|2019-10-01 00:56:29|2019-10-01 00:57:47|         264|         264|   NULL|                B00014|
|              B00014|2019-10-01 00:23:09|2019-10-01 00:28:27|         264|         264|   NULL|                B00014|
+--------------------+------------------

In [43]:
df_fhv_2019_10.schema

StructType([StructField('dispatching_base_num', StringType(), True), StructField('pickup_datetime', StringType(), True), StructField('dropOff_datetime', StringType(), True), StructField('PUlocationID', StringType(), True), StructField('DOlocationID', StringType(), True), StructField('SR_Flag', StringType(), True), StructField('Affiliated_base_number', StringType(), True)])

In [5]:
import pandas as pd

In [44]:
df_fhv_pd = pd.read_csv('fhv_2019_10/fhv_tripdata_2019-10.csv.gz', nrows=1000)

In [45]:
df_fhv_pd.dtypes

dispatching_base_num       object
pickup_datetime            object
dropOff_datetime           object
PUlocationID              float64
DOlocationID              float64
SR_Flag                   float64
Affiliated_base_number     object
dtype: object

In [46]:
from pyspark.sql import types

In [47]:
fhv_schema = types.StructType([
        types.StructField('dispatching_base_num', types.StringType(), True),
        types.StructField('pickup_datetime', types.TimestampType(), True), 
        types.StructField('dropOff_datetime', types.TimestampType(), True), 
        types.StructField('PUlocationID', types.IntegerType(), True), 
        types.StructField('DOlocationID', types.IntegerType(), True), 
        types.StructField('SR_Flag', types.StringType(), True), 
        types.StructField('Affiliated_base_number', types.StringType(), True)])

In [48]:
df_fhv = spark.read \
.option("header", "true") \
.schema(fhv_schema) \
.csv('fhv_2019_10')

In [49]:
df_fhv \
        .repartition(6) \
        .write.parquet('fhv_output', mode='overwrite')

In [50]:
df_fhv.registerTempTable('fhv_table')

/opt/homebrew/Cellar/apache-spark/3.5.0/libexec/python/pyspark/sql/dataframe.py:329: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [51]:
df_fhv_count_15th_Oct = spark.sql("""
SELECT 
    -- Reveneue grouping 
    COUNT(1) AS COUNT
FROM
    fhv_table
WHERE
    pickup_datetime >= '2019-10-15 00:00:00' AND pickup_datetime <= '2019-10-15 23:59:59'
""")

In [52]:
df_fhv_count_15th_Oct.show()

+-----+
|COUNT|
+-----+
|62610|
+-----+



In [53]:
from pyspark.sql import functions as F

In [62]:
df_fhv_hours = df_fhv.withColumn('hours_diff', (F.col('dropOff_datetime').cast('long') - F.col('pickup_datetime').cast('long'))/3600) 

In [63]:
df_fhv_hours.show()

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+--------------------+
|dispatching_base_num|    pickup_datetime|   dropOff_datetime|PUlocationID|DOlocationID|SR_Flag|Affiliated_base_number|          hours_diff|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+--------------------+
|              B00009|2019-10-01 00:23:00|2019-10-01 00:35:00|         264|         264|   NULL|                B00009|                 0.2|
|              B00013|2019-10-01 00:11:29|2019-10-01 00:13:22|         264|         264|   NULL|                B00013| 0.03138888888888889|
|              B00014|2019-10-01 00:11:43|2019-10-01 00:37:20|         264|         264|   NULL|                B00014| 0.42694444444444446|
|              B00014|2019-10-01 00:56:29|2019-10-01 00:57:47|         264|         264|   NULL|                B00014|0.021666666666666667|
|            

In [64]:
df_fhv_hours.registerTempTable('hours_fhv_table')

/opt/homebrew/Cellar/apache-spark/3.5.0/libexec/python/pyspark/sql/dataframe.py:329: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [65]:
df_fhv_longest_trip = spark.sql("""
SELECT 
    -- Reveneue grouping 
    hours_diff
FROM
    hours_fhv_table
    
ORDER BY hours_diff DESC
""")

In [66]:
df_fhv_longest_trip.show()

+------------------+
|        hours_diff|
+------------------+
|          631152.5|
|          631152.5|
| 87672.44083333333|
| 70128.02805555555|
|            8794.0|
| 8785.166666666666|
|1464.5344444444445|
|1057.8266666666666|
|1057.2705555555556|
| 794.5530555555556|
| 794.3858333333334|
|          794.2975|
| 793.9980555555555|
| 793.9883333333333|
| 793.8602777777778|
| 793.8108333333333|
|           793.785|
| 793.7694444444444|
| 793.7538888888889|
| 793.7463888888889|
+------------------+
only showing top 20 rows



In [67]:
df_zones = spark.read.parquet('zones')

In [68]:
df_fhv_zone = df_fhv_hours.join(df_zones, df_fhv_hours.PUlocationID == df_zones.LocationID)

In [72]:
df_fhv_zone.show()

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+--------------------+----------+-------+---------------+------------+
|dispatching_base_num|    pickup_datetime|   dropOff_datetime|PUlocationID|DOlocationID|SR_Flag|Affiliated_base_number|          hours_diff|LocationID|Borough|           Zone|service_zone|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+--------------------+----------+-------+---------------+------------+
|              B00009|2019-10-01 00:23:00|2019-10-01 00:35:00|         264|         264|   NULL|                B00009|                 0.2|       264|Unknown|             NV|         N/A|
|              B00013|2019-10-01 00:11:29|2019-10-01 00:13:22|         264|         264|   NULL|                B00013| 0.03138888888888889|       264|Unknown|             NV|         N/A|
|              B00014|2019-10-01 00:11:43|2019-10-01 00

In [73]:
df_fhv_zone.registerTempTable('fhv_zone_table')

/opt/homebrew/Cellar/apache-spark/3.5.0/libexec/python/pyspark/sql/dataframe.py:329: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [76]:
df_fhv_zone_least_popular = spark.sql("""
SELECT 
    Zone, COUNT(1) AS num_records
FROM
    fhv_zone_table
GROUP BY
    Zone
ORDER BY
    num_records
""")

In [77]:
df_fhv_zone_least_popular.show()

+--------------------+-----------+
|                Zone|num_records|
+--------------------+-----------+
|         Jamaica Bay|          1|
|Governor's Island...|          2|
| Green-Wood Cemetery|          5|
|       Broad Channel|          8|
|     Highbridge Park|         14|
|        Battery Park|         15|
|Saint Michaels Ce...|         23|
|Breezy Point/Fort...|         25|
|Marine Park/Floyd...|         26|
|        Astoria Park|         29|
|    Inwood Hill Park|         39|
|       Willets Point|         47|
|Forest Park/Highl...|         53|
|  Brooklyn Navy Yard|         57|
|        Crotona Park|         62|
|        Country Club|         77|
|     Freshkills Park|         89|
|       Prospect Park|         98|
|     Columbia Street|        105|
|  South Williamsburg|        110|
+--------------------+-----------+
only showing top 20 rows

